# What's in this exercise?
We run the common functions notebook so we can reuse capability defined there, and then...<BR>
1) Load yellow taxi data in staging directory to raw data directory, and save as parquet<BR> 
2) Create external unmanaged Hive tables<BR>
3) Create statistics for tables

#### 1. Declare necessary variables

In [2]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,LongType,FloatType,DoubleType, TimestampType

In [3]:
storageAccountName = dbutils.secrets.get(scope="databricks-warehouse", key="storage-name")
storageAccountKey = dbutils.secrets.get(scope="databricks-warehouse", key="storage-key")

In [4]:
spark.conf.set(f"fs.azure.account.key.{storageAccountName}.dfs.core.windows.net", storageAccountKey)

In [5]:
# Define source and destination directories
srcDataDirRoot = f"abfss://bronze@{storageAccountName}.dfs.core.windows.net/transactional-data/" #Root dir for source data
destDataDirRoot = f"abfss://silver@{storageAccountName}.dfs.core.windows.net/nyctaxi/transactions/yellow-taxi/" #Root dir for consumable data

#Canonical ordered column list for yellow taxi across years to homogenize schema
canonicalTripSchemaColList = ["taxi_type","vendor_id","pickup_datetime","dropoff_datetime","store_and_fwd_flag","rate_code_id","pickup_location_id","dropoff_location_id","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","passenger_count","trip_distance","fare_amount","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount","payment_type","trip_year","trip_month"]

#### 2. Define schema for source data
Different years have different schemas - fields added/removed

In [8]:
#Schema for data based on year and month

# 2017
yellowTripSchema2017H1 = StructType([
    StructField("vendor_id", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("rate_code_id", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("pickup_location_id", IntegerType(), True),
    StructField("dropoff_location_id", IntegerType(), True),
    StructField("payment_type", StringType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True)])

#Second half of 2016
yellowTripSchema2016H2 = StructType([
    StructField("vendor_id", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("rate_code_id", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("pickup_location_id", IntegerType(), True),
    StructField("dropoff_location_id", IntegerType(), True),
    StructField("payment_type", StringType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),
    StructField("junk1", StringType(), True),
    StructField("junk2", StringType(), True)])

# 2015 and 2016 first half of the year
yellowTripSchema20152016H1 = StructType([
    StructField("vendor_id", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("pickup_longitude", DoubleType(), True),
    StructField("pickup_latitude", DoubleType(), True),
    StructField("rate_code_id", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("dropoff_longitude", DoubleType(), True),
    StructField("dropoff_latitude", DoubleType(), True),
    StructField("payment_type", StringType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True)])

# 2009 though 2014
yellowTripSchemaPre2015 = StructType([
    StructField("vendor_id", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("pickup_longitude", DoubleType(), True),
    StructField("pickup_latitude", DoubleType(), True),
    StructField("rate_code_id", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("dropoff_longitude", DoubleType(), True),
    StructField("dropoff_latitude", DoubleType(), True),
    StructField("payment_type", StringType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("total_amount", DoubleType(), True)])

#### 3. Some functions

In [9]:
#1) Function to determine schema for a given year and month
#Input:  Year and month
#Output: StructType for applicable schema 
#Sample call: print getSchemaStruct(2009,1)

def getTaxiSchema(tripYear, tripMonth):
  taxiSchema = None

  if(tripYear > 2008 and tripYear < 2015):
    taxiSchema = yellowTripSchemaPre2015
  elif(tripYear == 2016 and tripMonth > 6):
    taxiSchema = yellowTripSchema2016H2
  elif((tripYear == 2016 and tripMonth < 7) or (tripYear == 2015)):
    taxiSchema = yellowTripSchema20152016H1
  elif(tripYear == 2017 and tripMonth < 7):
    taxiSchema = yellowTripSchema2017H1
  
  return taxiSchema

In [10]:
#2) Function to add columns to dataframe as required to homogenize schema
#Input:  Dataframe, year and month
#Output: Dataframe with homogenized schema 
#Sample call: println(getSchemaHomogenizedDataframe(DF,2014,6))

from pyspark.sql.functions import *

def getSchemaHomogenizedDataframe(sourceDF,tripYear, tripMonth):
  if(tripYear > 2008 and tripYear < 2015):
    sourceDF = (sourceDF.withColumn("pickup_location_id", lit(0).cast("integer"))
              .withColumn("dropoff_location_id", lit(0).cast("integer"))
              .withColumn("improvement_surcharge",lit(0).cast("double"))
              .withColumn("junk1",lit(""))
              .withColumn("junk2",lit(""))
              .withColumn("trip_year",substring(col("pickup_datetime"),0, 4))
              .withColumn("trip_month",substring(col("pickup_datetime"),6,2))
              .withColumn("taxi_type",lit("yellow"))
              .withColumn("temp_pickup_longitude", col("pickup_longitude").cast("string"))
                                      .drop("pickup_longitude").withColumnRenamed("temp_pickup_longitude", "pickup_longitude")
              .withColumn("temp_dropoff_longitude", col("dropoff_longitude").cast("string"))
                                      .drop("dropoff_longitude").withColumnRenamed("temp_dropoff_longitude", "dropoff_longitude")
              .withColumn("temp_pickup_latitude", col("pickup_latitude").cast("string"))
                                      .drop("pickup_latitude").withColumnRenamed("temp_pickup_latitude", "pickup_latitude")
              .withColumn("temp_dropoff_latitude", col("dropoff_latitude").cast("string"))
                                      .drop("dropoff_latitude").withColumnRenamed("temp_dropoff_latitude", "dropoff_latitude")
              .withColumn("temp_payment_type", col("payment_type").cast("string")).drop("payment_type").withColumnRenamed("temp_payment_type", "payment_type"))
  elif((tripYear == 2016 and tripMonth < 7) or (tripYear == 2015)):
    sourceDF = (sourceDF.withColumn("pickup_location_id", lit(0).cast("integer"))
              .withColumn("dropoff_location_id", lit(0).cast("integer"))
              .withColumn("junk1",lit(""))
              .withColumn("junk2",lit(""))
              .withColumn("trip_year",substring(col("pickup_datetime"),0, 4))
              .withColumn("trip_month",substring(col("pickup_datetime"),6,2))
              .withColumn("taxi_type",lit("yellow"))
              .withColumn("temp_vendor_id", col("vendor_id").cast("string")).drop("vendor_id").withColumnRenamed("temp_vendor_id", "vendor_id")
              .withColumn("temp_pickup_longitude", col("pickup_longitude").cast("string"))
                                      .drop("pickup_longitude").withColumnRenamed("temp_pickup_longitude", "pickup_longitude")
              .withColumn("temp_dropoff_longitude", col("dropoff_longitude").cast("string"))
                                      .drop("dropoff_longitude").withColumnRenamed("temp_dropoff_longitude", "dropoff_longitude")
              .withColumn("temp_pickup_latitude", col("pickup_latitude").cast("string"))
                                      .drop("pickup_latitude").withColumnRenamed("temp_pickup_latitude", "pickup_latitude")
              .withColumn("temp_dropoff_latitude", col("dropoff_latitude").cast("string"))
                                      .drop("dropoff_latitude").withColumnRenamed("temp_dropoff_latitude", "dropoff_latitude")
              .withColumn("temp_payment_type", col("payment_type").cast("string")).drop("payment_type").withColumnRenamed("temp_payment_type", "payment_type"))
  elif(tripYear == 2016 and tripMonth > 6):
    sourceDF = (sourceDF.withColumn("pickup_longitude", lit(""))
              .withColumn("pickup_latitude", lit(""))
              .withColumn("dropoff_longitude", lit(""))
              .withColumn("dropoff_latitude", lit(""))
              .withColumn("trip_year",substring(col("pickup_datetime"),0, 4))
              .withColumn("trip_month",substring(col("pickup_datetime"),6,2))
              .withColumn("taxi_type",lit("yellow"))
              .withColumn("temp_vendor_id", col("vendor_id").cast("string")).drop("vendor_id").withColumnRenamed("temp_vendor_id", "vendor_id")
              .withColumn("temp_payment_type", col("payment_type").cast("string")).drop("payment_type").withColumnRenamed("temp_payment_type", "payment_type"))
  elif(tripYear == 2017 and tripMonth < 7):
    sourceDF = (sourceDF.withColumn("pickup_longitude", lit(""))
              .withColumn("pickup_latitude", lit(""))
              .withColumn("dropoff_longitude", lit(""))
              .withColumn("dropoff_latitude", lit(""))
              .withColumn("trip_year",substring(col("pickup_datetime"),0, 4))
              .withColumn("trip_month",substring(col("pickup_datetime"),6,2))
              .withColumn("taxi_type",lit("yellow"))
              .withColumn("junk1",lit(""))
              .withColumn("junk2",lit(""))
              .withColumn("temp_vendor_id", col("vendor_id").cast("string")).drop("vendor_id").withColumnRenamed("temp_vendor_id", "vendor_id")
              .withColumn("temp_payment_type", col("payment_type").cast("string")).drop("payment_type").withColumnRenamed("temp_payment_type", "payment_type"))
  else:
    sourceDF
    
  return sourceDF

#### 4. Read CSV, homogenize schema across years, save as parquet

In [ ]:

# To make Hive Parquet format compatible with Spark Parquet format
spark.conf.set("spark.sql.parquet.writeLegacyFormat", "true")

# Disable delta optimizeWrite for consistent number of partition (when write from 2nd time, spark with delta format can reduce number of partitions)
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "false")

#Process data, save as parquet
for j in range(2009,2018):
    
  endMonth = None
  if (j==2017):
    endMonth = 6 
  else: endMonth = 12
  
  for i in range(1,endMonth+1):
    
    srcDataFile= "{}year={}/month={:02d}/type=yellow/yellow_tripdata_{}-{:02d}.csv".format(srcDataDirRoot,j,i,j,i)
    print("Year={}; Month={}".format(j,i))
    print(srcDataFile)

    #Source schema
    taxiSchema = getTaxiSchema(j,i)

    #Read source data
    taxiDF = (spark.read.format("csv")
                    .option("header", True)
                    .schema(taxiSchema)
                    .option("delimiter",",")
                    .load(srcDataFile).cache())

    #Add additional columns to homogenize schema across years
    taxiFormattedDF = getSchemaHomogenizedDataframe(taxiDF, j, i)

    #Order all columns to align with the canonical schema for yellow taxi
    taxiCanonicalDF = taxiFormattedDF.select(*canonicalTripSchemaColList)
    
    # year_month_dir = "{}trip_year={}/trip_month={:02d}".format(destDataDirRoot,j,i)
    # dbutils.fs.rm(year_month_dir,recurse=True)
    
    taxiCanonicalDF.repartition(4).write.option("compression", "zstd").format("delta").mode("append").partitionBy("trip_year","trip_month").save(destDataDirRoot)


In [11]:
def create_table(schema: str, table_name: str, parquet_dir: str, location: str):
    spark.sql(f"use {schema};")
    spark.sql(f"DROP TABLE IF EXISTS {table_name};")
    spark.sql(f"CREATE TABLE IF NOT EXISTS {table_name} USING delta LOCATION '{location}/{parquet_dir}';")

In [12]:
create_table(schema="synapse_nyc_reference.nyctaxi", table_name="yellow_taxi_trips_raw", parquet_dir="", location=destDataDirRoot)

In [ ]:
#%sql
#FSCK REPAIR TABLE nyctaxi.yellow_taxi_trips_raw;

<Empty result set>

#### 5. Statistics table

In [16]:
%sql
select * from nyctaxi.yellow_taxi_trips_raw;

taxi_type,vendor_id,pickup_datetime,dropoff_datetime,store_and_fwd_flag,rate_code_id,pickup_location_id,dropoff_location_id,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_year,trip_month
yellow,CMT,2009-01-02T22:08:52Z,2009-01-02T22:14:20Z,null,null,0,0,-73.988629,40.718825,-74.003554,40.725074,3,1,5.8,0,null,1,0,0,6.8,Credit,2009,01
yellow,DDS,2009-01-17T17:39:56Z,2009-01-17T17:44:02Z,null,null,0,0,-73.983576,40.721534,-73.996676,40.725463,1,0.7,4.5,0,null,0,0,0,4.5,CASH,2009,01
yellow,VTS,2009-01-12T10:00:00Z,2009-01-12T10:15:00Z,null,null,0,0,-73.968885,40.786082,-73.96796,40.760827,1,2.33,10.5,0,null,0,0,0,10.5,CASH,2009,01
yellow,CMT,2009-01-30T00:01:08Z,2009-01-30T00:24:03Z,null,null,0,0,-73.992364,40.735524,-73.956625,40.685723,1,5.4,17.4,0,null,0,0,0,17.4,Cash,2009,01
yellow,VTS,2009-01-24T18:21:00Z,2009-01-24T18:32:00Z,null,null,0,0,-73.966707,40.761563,-73.990032,40.756758,2,1.95,8.1,0,null,0,0,0,8.1,CASH,2009,01
yellow,CMT,2009-01-07T08:41:04Z,2009-01-07T09:01:55Z,null,null,0,0,-73.978916,40.772521,-74.01528,40.714393,1,5.4,15.7,0,null,2.35,0,0,18.05,Credit,2009,01
yellow,VTS,2009-01-21T15:23:00Z,2009-01-21T15:31:00Z,null,null,0,0,-73.982902,40.7478,-73.97847,40.761495,1,1.34,6.5,0,null,0,0,0,6.5,CASH,2009,01
yellow,VTS,2009-01-21T19:11:00Z,2009-01-21T19:14:00Z,null,null,0,0,-73.992638,40.742995,-73.986095,40.752093,1,0.73,4.1,1,null,0,0,0,5.1,CASH,2009,01
yellow,CMT,2009-01-11T11:03:15Z,2009-01-11T11:08:57Z,null,null,0,0,-73.966599,40.76758,-73.980044,40.775892,1,1.3,5.7,0,null,0,0,0,5.7,Cash,2009,01
yellow,CMT,2009-01-23T01:42:37Z,2009-01-23T01:50:09Z,null,null,0,0,-73.983498,40.7507,-74.002978,40.727114,1,2,7.8,0,null,0,0,0,7.8,Cash,2009,01


In [28]:
%sql
select COUNT(1) from nyctaxi.yellow_taxi_trips_raw;

count(1)
1369889765


In [30]:
%sql
DESCRIBE HISTORY nyctaxi.yellow_taxi_trips_raw;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
6,2025-03-25T22:48:59Z,2426707124863848,nhudinhpoc@outlook.com,WRITE,[object Object],null,null,0320-050324-uw98cg7q,5,WriteSerializable,false,[object Object],null,Databricks-Runtime/16.2.x-cpu-ml-scala2.12
5,2025-03-25T22:44:05Z,2426707124863848,nhudinhpoc@outlook.com,WRITE,[object Object],null,null,0320-050324-uw98cg7q,4,WriteSerializable,false,[object Object],null,Databricks-Runtime/16.2.x-cpu-ml-scala2.12
4,2025-03-25T22:29:38Z,2426707124863848,nhudinhpoc@outlook.com,FSCK,[object Object],null,null,0320-050324-uw98cg7q,3,SnapshotIsolation,false,[object Object],null,Databricks-Runtime/16.2.x-cpu-ml-scala2.12
3,2025-03-25T22:27:59Z,2426707124863848,nhudinhpoc@outlook.com,WRITE,[object Object],null,null,0320-050324-uw98cg7q,2,WriteSerializable,true,[object Object],null,Databricks-Runtime/16.2.x-cpu-ml-scala2.12
2,2025-03-25T22:22:20Z,2426707124863848,nhudinhpoc@outlook.com,WRITE,[object Object],null,null,0320-050324-uw98cg7q,1,WriteSerializable,false,[object Object],null,Databricks-Runtime/16.2.x-cpu-ml-scala2.12
1,2025-03-25T22:13:36Z,2426707124863848,nhudinhpoc@outlook.com,WRITE,[object Object],null,null,0320-050324-uw98cg7q,0,WriteSerializable,false,[object Object],null,Databricks-Runtime/16.2.x-cpu-ml-scala2.12
0,2025-03-25T22:07:51Z,2426707124863848,nhudinhpoc@outlook.com,CONVERT,[object Object],null,null,0320-050324-uw98cg7q,-1,Serializable,false,[object Object],null,Databricks-Runtime/16.2.x-cpu-ml-scala2.12
